In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from utils.config_utils import get_config
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
import warnings
from sklearn.model_selection import StratifiedKFold

In [17]:
def get_data_set_with_certain_group_excluded(sensitive_variable_name, sensitive_variable_map, original_dataset):
    # return a dictionary of dataframes, each dataframe is a dataset with certain group excluded
    # the key is the excluded group
    result_dic = {}
    for key, value in sensitive_variable_map.items():
        train_set = original_dataset[original_dataset[sensitive_variable_name] == value] 
        #get not equal to value
        test_set = original_dataset[original_dataset[sensitive_variable_name] != value]
        result_dic[key] = {
            'train_set': train_set,
            'test_set': test_set
        }
    return result_dic


def get_data_set_original(dataset, config):
    warnings.warn("Data leakage problem", DeprecationWarning)
    train_set, test_set = train_test_split(dataset, test_size=config['test_size'], random_state=config['random_state'])
    return train_set, test_set


def split_dataset_by_map(sensitive_variable_name, sensitive_variable_map, original_dataset, test_size = 0.4):
    warnings.warn("Data leakage problem", DeprecationWarning)
    result_dic = {}
    for key, value in sensitive_variable_map.items():
        dataset = original_dataset[original_dataset[sensitive_variable_name] == value] 
        train_set, test_set = train_test_split(dataset, test_size=test_size)
        result_dic[key] = {
            'train_set': train_set,
            'test_set': test_set
        }
    return result_dic


def get_model(name, params):

    if name == 'lr':
        model = LogisticRegression(**params)
    elif name == 'svm':
        model = SVC(**params)
    elif name == 'rf':
        model = RandomForestClassifier(**params)
    elif name == 'gb':
        model = GradientBoostingClassifier(**params)
    elif name == 'nn':
        model = MLPClassifier(**params)
    else:
        raise ValueError('No such model')
    return model


def preprocess_data(features, config):
    
    features = features.copy()
    drop_columns = config['drop_columns']
    # drop the columns
    features = features.drop(drop_columns, axis=1)
    # get the missing values threshold
    missing_values_threshold = config['missing_values_threshold']
    #calculate missing values ratio
    missing_ratio = features.isnull().mean()
    # print(f'missing_ratio: {missing_ratio}')
    #retain the values which are less than the threshold
    variables_to_be_retained = features.columns[missing_ratio <= missing_values_threshold]
    features = features[variables_to_be_retained]
    # The samplest way to fill missing numerical values is to fill them with 0
    features = features.fillna(0)
    # check if there are any null values
    has_null_values = features.isnull().any().any()
    # if has_null_values:
    #     print("has null values.")
    # else:
    #     print("has no null values.")

    string_columns = features.select_dtypes(include='object').columns
    # print the variables that are numerical(not strings)
    num_string_columns = len(string_columns)
    # print(f"String Columns in Dataframe are are listed below. There are {num_string_columns} colums in total.")

    # for col in string_columns:
    #     print(col)
    # drop the string columns
    features = features.drop(string_columns, axis=1)
    # One-Hot Encoding
    features = pd.get_dummies(features)
    # normalize
    normalize_method = config['normalize_method']
    if normalize_method == 'min_max':
        features = (features - features.min()) / (features.max() - features.min())
    elif normalize_method == 'z_score':
        features = (features - features.mean()) / features.std()
    else:
        raise ValueError('No such normalize method')
    # There maybe some identical columns in the dataset, so we need to remove them. (Maybe there are other ways to do this)
    columns_with_null = features.columns[features.isnull().any()]

    # Print the column names with null values
    # print("Columns with null values:")
    # print(columns_with_null)
    features.drop(columns_with_null, axis=1, inplace=True)
    
    return features



def calculate_classification_metrics(y_true, y_pred_prob):
    # Threshold 0,5 by default
    y_pred = np.where(y_pred_prob >= 0.5, 1, 0)
    
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Recall
    sensitivity = recall_score(y_true, y_pred)

    # Specificity
    specificity = recall_score(y_true, y_pred, pos_label=0)

    # F1 Score
    f1 = f1_score(y_true, y_pred)

    # ROC AUC
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    # FPR FNR
    false_positive_rate = fpr[1]
    false_negative_rate = 1 - tpr[1]

    # confusion_matrix
    conf_matrix = confusion_matrix(y_true, y_pred)

    result_dic = {
        "Accuracy": accuracy,
        "Sensitivity (Recall)": sensitivity,
        "Specificity": specificity,
        "F1 Score": f1,
        "ROC AUC": roc_auc,
        "False Positive Rate": false_positive_rate,
        "False Negative Rate": false_negative_rate,
    }
    
    return result_dic, (fpr, tpr), conf_matrix


def plot_auc(metrics, tag, save_fig = True):
    plt.figure(figsize=(8, 6))
    for model_name, metric in metrics.items():
        fpr, tpr = metric['fpr_tpr']
        roc_auc = metric['ROC AUC']
        plt.plot(fpr, tpr, label=f'{model_name} (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic {tag}')
    plt.legend(loc='lower right')
    path = os.path.join("pics", f'roc_curve{tag}.png')
    if save_fig :
        plt.savefig(path)


def stratified_group_kfold(df, config):
    """
    Perform stratified sampling similar to StratifiedGroupKFold, while ensuring test set size is close to the specified test_size ratio.
    
    Returns:
    train_indices, test_indices: Lists of training and testing indices.
    """
    target_col = config['target_variable']['name']

    # Get the group variable for stratified sampling
    group_col = config['groups']
    # Get the random seed
    random_state = config['random_state']
    # Get the k_fold
    k_fold = config['k_fold']
    # Get the test_size
    test_size = config['test_size']
    X = df.drop(columns=[target_col, group_col]).values
    y = df[target_col].values
    groups = df[group_col].values
    
    stratified_kfold = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=random_state)
    train_indices, test_indices = [], []
    
    for train_idx, test_idx in stratified_kfold.split(X, y):
        X_train, X_test, y_train, y_test, groups_train, groups_test = \
            X[train_idx], X[test_idx], y[train_idx], y[test_idx], groups[train_idx], groups[test_idx]
        
        # Calculate the target size for the test set
        target_test_size = int(len(X) * test_size)
        
        # If the test set size is less than the target size, move additional samples from the train set to the test set
        if len(test_idx) < target_test_size:
            additional_samples = target_test_size - len(test_idx)
            rng = np.random.default_rng(random_state)
            additional_samples_indices = rng.choice(len(train_idx), additional_samples, replace=False)
            
            test_idx = np.concatenate((test_idx, train_idx[additional_samples_indices]))
            train_idx = np.delete(train_idx, additional_samples_indices)
        
        train_indices.append(train_idx)
        test_indices.append(test_idx)
    X = df.drop(columns=[target_col, group_col]).copy()
    y = df[target_col].copy()
    
    return train_indices, test_indices, X, y


def split_dataset_by_sensitive_variable(protected_attribute_name, protected_attribute_map, original_dataset, config):
    #split the dataset by sensitive variable
    # 
    original_dataset = original_dataset.copy()
    target_col = config['target_variable']['name']
    # Get the group variable for stratified sampling
    group_col = config['groups']
    # Get the random seed
    random_state = config['random_state']
    # Get the k_fold
    k_fold = config['k_fold']
    # Get the test_size
    test_size = config['test_size']

    X = original_dataset.drop(columns=[target_col, group_col, protected_attribute_name])
    X = preprocess_data(X, config['preprocessing'])
    y = original_dataset[target_col]
    protected_attribute = original_dataset[protected_attribute_name]
    groups = original_dataset[group_col]
    original_dataset = pd.concat([X, y, protected_attribute, groups], axis=1)
    result_dic = {}

    for key, value in protected_attribute_map.items():

        dataset = original_dataset[original_dataset[protected_attribute_name] == value].copy()
        train_indices, test_indices, X, y = stratified_group_kfold(df = dataset, config=config)
        result_dic[key] = {
            'train_set_indices': train_indices,
            'test_set_indices': test_indices,
            'X': X,
            'y': y
        }
    
    return result_dic


def get_per_sensitive_variable_split(protected_attributes, original_dataset, config):
    datasets_dic = {}
    for protected_attribute in protected_attributes.values():

        protected_attribute_name = protected_attribute['name']
        protected_attribute_map = protected_attribute['map']
        datasets_dic[protected_attribute_name] = split_dataset_by_sensitive_variable(protected_attribute_name = protected_attribute_name,
                                                                                            protected_attribute_map=protected_attribute_map, 
                                                                                            original_dataset = original_dataset,
                                                                                            config=config)
    return datasets_dic


def train_and_evaluate_one_model(model, dataset_dic, config):
    target_variable = config['target_variable']['name']
    target_variable_map = config['target_variable']['map']
    #metrics dic for all classes
    result_dic = {}
    k_fold = config['k_fold']
    
    for c_train, dic_train in dataset_dic.items():
        
        result_dic[c_train] = {}
        # metrics dic for a single class
        for fold in range(k_fold):
            X = dic_train['X'].copy()
            y = dic_train['y'].copy()
            y = y.map(target_variable_map)
            print(f"Training and Testing on fold {fold} for class {c_train}")
            # Get the train set indice for the current fold
            train_set_indice = dic_train['train_set_indices'][fold]
            X_train, y_train = X.iloc[train_set_indice], y.iloc[train_set_indice]
            model.fit(X_train, y_train)
            for c_test, dic_test in dataset_dic.items():
                # print(f"Testing on class {c_test}")
                X = dic_test['X'].copy()
                y = dic_test['y'].copy()
                y = y.map(target_variable_map)
                # Get the test set indice for the current fold
                test_set_indice = dic_test['test_set_indices'][fold]
                X_test, y_test = X.iloc[test_set_indice], y.iloc[test_set_indice]
                y_pred_prob = model.predict_proba(X_test)[:, 1]
                #calculate metrics
                metrics, _, _ = calculate_classification_metrics(y_true=y_test, y_pred_prob=y_pred_prob)
                # acc = metrics['Accuracy']
                # print(f'{c_train} vs {c_test}: {acc}')
                if  not c_test in result_dic[c_train]:
                    result_dic[c_train][c_test] = metrics
                else :
                    for key, value in metrics.items():
                        result_dic[c_train][c_test][key] += value
                        
        # Get the average of the metrics
        for c_test, dic in dataset_dic.items():
            for key, value in result_dic[c_train][c_test].items():
                result_dic[c_train][c_test][key] = value / k_fold

    return result_dic


def train_and_evaluate_models(models_dict, datasets_dic, config):
    
    result_dic = {}

    for model_name, model_params in models_dict.items():

        result_dic[model_name] = {}
        
        for protected_attribute_name, dataset_dic in datasets_dic.items():

            result_dic[model_name][protected_attribute_name] = {}
            model = get_model(model_name, model_params)
            result_dic[model_name][protected_attribute_name] = train_and_evaluate_one_model(model=model, dataset_dic=dataset_dic, config=config)
        
    return result_dic


def result_dic2csv(result_dic, metric_name):
    metrics = ['Accuracy', 'Sensitivity (Recall)', 'Specificity', 'F1 Score', 'ROC AUC', 'False Positive Rate', 'False Negative Rate']
    if metric_name not in metrics:
        raise ValueError('No such metric')
    result_dic = result_dic.copy()
    
    for protected_attribute, protected_attribute_dic in result_dic.items():
        
        metrics = {}
        for c_1, dic in protected_attribute_dic.items():
            
            metrics[c_1] = {}
            
            for c_2, metrics_ in dic.items():

                metrics[c_1][c_2] = metrics_[metric_name]

        df = pd.DataFrame.from_dict(metrics, orient='index')
        df.to_csv(f'{protected_attribute}_{metric_name}.csv')


In [5]:
# train on one group and evaluate on all the groups
# e.g. train on trainning dataset of White people and evaluate on White, black, Asian, etc.

# The path of NACC Dataset
NACC_DATASET_PATH = os.path.join("data", "NACC.csv")
# The path of the Config file 
DATA_CONFIG_PATH = os.path.join("config", "config_default.yaml")
# Load the config file
config = get_config(DATA_CONFIG_PATH)
# Load the NACC dataset
NACC_dataset = pd.read_csv(NACC_DATASET_PATH, low_memory=False)
# Get the model dictionary
models_dict = config['model_dict']
# Get the protected attributes
protected_attributes = config['protected_attributes']

    

In [6]:
# Split the dataset by sensitive variable
datasets_dic = get_per_sensitive_variable_split(protected_attributes = protected_attributes,
                                                original_dataset=NACC_dataset.copy(),
                                                config=config)
# Train and evaluate the models
result_dic = train_and_evaluate_models(models_dict, datasets_dic, config)
# Save the result to csv
result_dic2csv(result_dic, 'ROC AUC')
result_dic2csv(result_dic, 'Accuracy')

Training and Testing on fold 0 for class White
Training and Testing on fold 1 for class White
Training and Testing on fold 2 for class White
Training and Testing on fold 3 for class White
Training and Testing on fold 0 for class black_or_African_American
Training and Testing on fold 1 for class black_or_African_American
Training and Testing on fold 2 for class black_or_African_American
Training and Testing on fold 3 for class black_or_African_American
Training and Testing on fold 0 for class American_Indian_or_Alaska_Native
Training and Testing on fold 1 for class American_Indian_or_Alaska_Native
Training and Testing on fold 2 for class American_Indian_or_Alaska_Native
Training and Testing on fold 3 for class American_Indian_or_Alaska_Native
Training and Testing on fold 0 for class Asian
Training and Testing on fold 1 for class Asian
Training and Testing on fold 2 for class Asian
Training and Testing on fold 3 for class Asian
Training and Testing on fold 0 for class Male
Training and T